# Step 1: Load the Modules

In [1]:
# Load Python Modules
import sys
import os
import time
from imp import reload
import threading
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt

# Set the working directory to the AutomatedCE folder so we have access to L1-L4 modules
# This assumes we are running from the testing folder
resp = os.getcwd()
if 'testing' in resp[-7:]:
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
    print(f"new directory is: {os.getcwd()}")

# Load Custom Modules
import testing.inputs  
reload(testing.inputs)
from testing.inputs import *
from L4 import DisplayWindows
from L4 import FileIO
from L4 import image_util
from L3 import SystemsBuilder
reload(SystemsBuilder)
reload(DisplayWindows)

new directory is: C:\Users\Luke\Desktop\Barracuda\AutomatedCE


C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L1\MicroControlServer.py


 Change System Config setting if thorlabs is needed. 


<module 'L4.DisplayWindows' from 'C:\\Users\\Luke\\Desktop\\Barracuda\\AutomatedCE\\L4\\DisplayWindows.py'>

In [2]:
%matplotlib qt

# Step 2 Load the Instrument Configuration

In [3]:
# Load the Hardware Componenets 
# Shouldnt have to change anything in here 
CONFIG_PATH = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\ParallelChip.cfg"

ce_system = SystemsBuilder.CESystem()
ce_system.load_config(CONFIG_PATH)
ce_system.open_controllers()
ce_system.startup_utilities()

daq
uniphase
{'xy_stage': <L2.XYControl.PycromanagerXY object at 0x00000243BF9901C8>, 'objective': <L2.ZControl.PycromanagerZ object at 0x0000024387C3CE08>, 'filter_wheel': <L2.FilterWheelControl.PycromanagerFilter object at 0x00000243BF9902C8>, 'excitation_shutter': <L2.ShutterControl.PycromanagerShutter object at 0x00000243BF8F4B88>, 'camera': <L2.CameraControl.PycromanagerControl object at 0x00000243BF9903C8>, 'high_voltage': <L2.HighVoltageControl.SpellmanPowerSupply object at 0x00000243BF8F4AC8>, 'lysis_laser': <L2.LaserControl.Uniphase object at 0x00000243BF9907C8>}
OPENING:  mmc1 <L1.Controllers.PycromanagerController object at 0x00000243BF740BC8> using port: pycromanager
OPENING:  daq1 <L1.DAQControllers.NiDaq object at 0x00000243BF746148>
Starting up: xy_stage, <L2.XYControl.PycromanagerXY object at 0x00000243BF9901C8>
Starting up: objective, <L2.ZControl.PycromanagerZ object at 0x0000024387C3CE08>
Starting up: filter_wheel, <L2.FilterWheelControl.PycromanagerFilter object at 

# Step 3 Enter Separation Parameters

In [6]:
# CHANGE PARAMETERS ONLY HERE
VOLTAGE = 2 # Specify in KILOVolts the voltage
TIME = 220 # Specify in seconds to run the electrophoresis
EXPOSURE =  250 # Specify in milliseconds camera exposure
DATA_FOLDER = r"E:\Ming\20201112" # Make sure r is in front of the quotation ex: r"E:\path\to\dir", Use the E drive
BINNING = 4 # Specify the camera binning to use, acceptable values are 1,2,4,8
DELAY = 0 # Specify time to wait before applying voltage
STOP_EVENT = threading.Event() # Use the set method to stop during a run 

# Step 4 Run Separation

In [7]:
# Don't Change this unless you know what you are doing:) 

def chip_run():
    im = image_util.ImageSaver(ce_system,data_folder=DATA_FOLDER)
    ce_system.excitation_shutter.set_auto_on()
    # AdjustCamera
    ce_system.camera.stop()
    ce_system.camera.set_exposure(EXPOSURE)
    try:
        ce_system.camera.set_binning(BINNING)
    except:
        pass
    im.add_callback()

    #Adjustvoltage
    ce_system.high_voltage.set_voltage(VOLTAGE)

    # Start Camera
    ce_system.camera.continuous_snap()

    #Delay
    time.sleep(DELAY)

    #Start Voltage
    ce_system.high_voltage.start()
    start_time= time.time()
    print("VOLTAGE ON")

    # wait for the run to finish
    while (time.time()-start_time)<TIME and not STOP_EVENT.is_set():
        time.sleep(0.05)


    ce_system.high_voltage.stop()
    im.remove_callback()
    ce_system.excitation_shutter.set_auto_off()
    im.save_image(timestamp=True, unique_folder=True)
    print("FINISHED")
STOP_EVENT.clear()
threading.Thread(target=chip_run).start()

HEYYOO
VOLTAGE ON


C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-31-41 266271.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-31-41 520618.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-31-41 776908.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-31-42 041224.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-31-42 261612.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L

C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-32-01 339627.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-32-01 552017.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-32-01 787392.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-32-01 995840.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-32-02 227242.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L

C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-32-29 640896.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-32-29 854356.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-32-30 084712.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-32-30 346012.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-32-30 577484.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L

C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-32-57 961156.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-32-58 195536.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-32-58 413947.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-32-58 661367.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-32-58 893666.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L

...

C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-34-18 597542.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-34-18 816911.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-34-19 056276.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-34-19 294633.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-34-19 513052.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L

C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-34-39 010898.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-34-39 248267.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-34-39 468711.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-34-39 729000.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-34-39 980383.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L

C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-34-59 073246.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-34-59 319605.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-34-59 561936.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-34-59 796313.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-35-00 013727.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L

C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-35-20 141916.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-35-20 356325.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-35-20 583714.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 16-35-20 840027.tiff is a low contrast image
  io.imsave(data_file, image)


FINISHED


###  Run the following when you need to Stop The Separation Immediately

In [13]:
STOP_EVENT.set()

C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 14-11-07 579854.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 14-11-07 818150.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 14-11-08 048536.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 14-11-08 305847.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 14-11-08 520269.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L

C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 14-11-25 178738.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 14-11-25 429131.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 14-11-25 671420.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 14-11-25 906774.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 14-11-26 127205.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L

C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 14-11-54 011641.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 14-11-54 237005.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 14-11-54 483348.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 14-11-54 727746.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 14-11-54 968099.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L

C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 14-12-22 979134.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 14-12-23 221487.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 14-12-23 450961.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 14-12-23 671302.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\image_util.py:109: UserWarning: E:\Ming\20201112\_00003\img 20-11-13 14-12-23 901664.tiff is a low contrast image
  io.imsave(data_file, image)
C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L

FINISHED


# APPENDIX A Lysis Laser

First, initialize the laser. Once initialized, turn on the switch at the laser head. 

Second, enter standby mode, at this stage the laser is ready to fire. Make sure curtains are lowered before entering standby mode. 

Third, Fire the laser 

Fourth, Enter Shutdown Sequence

In [3]:
from inputs import *

In [4]:
resp = get_yes_no("Do you really want to initilialize the Laser?")
if resp:
    las= ce_system.lysis_laser
    las.laser_stop()
    time.sleep(0.15)
else:
    ce_system.lysis_laser.stop()

Do you really want to Start the Laser? y


In [79]:
resp = get_yes_no("Do you want to enter 'STANDBY' mode for the lysis laser?")
if resp:
    las.stop()
    time.sleep(0.1)
    las.laser_standby()
    time.sleep(0.1)
    las.stop()
    time.sleep(0.1)
    las.laser_standby()


Do you want to enter 'STANDBY' mode for the lysis laser? y


In [82]:
resp = get_yes_no("Do you really want to fire the laser?")
if resp:
    las.laser_fire()


Do you really want to fire the laser? y


In [83]:
las.stop()

# Appendix B: Move the Objective

In [14]:
# Moves to position in Millimeters
ce_system.objective.set_z(0)

In [4]:
ce_system.camera._dev_name

'QCamera'

In [7]:
img = ce_system.camera.snap()

In [9]:
plt.imshow(img)

In [8]:
import numpy as np

In [9]:
l = np.zeros(12582912)

In [8]:
img.shape

(1536, 2048, 4)

In [13]:
l.reshape(1536,2048,4)

array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       ...,

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0.

In [16]:
ce_system.stop()

AttributeError: 'CESystem' object has no attribute 'stop'